In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import keras

In [2]:
# Define path to the dataset
PATH = "C:/Users/emanu/Universidade/Mestrado/1_Ano/2_Semestre/ACA/Projeto/ACA-Project/AML-Project2/dataset"

# Read the CSV file containing the image paths and labels
# Read the training set data
train_data = np.load(os.path.join(PATH, "trainX.npy"))
train_labels = np.load(os.path.join(PATH, "trainy.npy"), allow_pickle=True)

test_data = np.load(os.path.join(PATH, "testX.npy"))

print(train_data.shape)
display(train_data)

(12000, 50, 50, 3)


array([[[[ 47,  64,  77],
         [ 43,  57,  80],
         [182, 206, 224],
         ...,
         [ 10,   8,   8],
         [  9,   7,   7],
         [  7,   5,   5]],

        [[ 42,  59,  80],
         [ 14,  22,  35],
         [ 50,  62,  74],
         ...,
         [ 14,   9,  10],
         [ 10,   5,   6],
         [ 11,   6,   7]],

        [[ 51,  68,  87],
         [  0,   5,   9],
         [  7,  14,  23],
         ...,
         [ 13,   8,  10],
         [ 13,   8,   9],
         [ 11,   6,   7]],

        ...,

        [[ 57,  63,  76],
         [ 43,  55,  65],
         [ 33,  45,  63],
         ...,
         [ 37,  36,  40],
         [ 17,  12,  14],
         [ 16,  11,  13]],

        [[ 48,  58,  65],
         [ 47,  53,  66],
         [ 45,  53,  60],
         ...,
         [ 19,  21,  22],
         [ 25,  20,  22],
         [ 14,   9,  10]],

        [[ 32,  40,  47],
         [ 13,  19,  30],
         [ 11,  17,  24],
         ...,
         [237, 239, 247],
        

In [ ]:
rescale = tf.keras.Sequential([
  layers.Rescaling(1./255)
])

In [9]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

# Add the image to a batch.
image = tf.expand_dims(train_data, 0)

ModuleNotFoundError: No module named 'tensorflow_federated'

In [ ]:
model = tf.keras.Sequential([
  # Add the preprocessing layers you created earlier.
  rescale,
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  # Rest of your model.
])

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

In [174]:
model = models.Sequential()

model.add(layers.Conv2D(32, 3, activation='relu', input_shape=(50, 50, 3)))
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(128, 3, activation='relu'))
model.add(layers.MaxPool2D())
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 48, 48, 32)        896       
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 24, 24, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 22, 22, 64)        18496     
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 11, 11, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_42 (Conv2D)          (None, 9, 9, 128)         73856     
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 4, 4, 128)      

In [175]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

In [176]:
train_Y_one_hot = keras.utils.to_categorical(y_train)

In [177]:
model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x=X_train,
    y=train_Y_one_hot,
    batch_size=64,
    epochs=40,
    validation_split=0.1
)

Epoch 1/40
135/135 [==============================] - 8s 51ms/step - loss: 1.2614 - accuracy: 0.4684 - val_loss: 0.9830 - val_accuracy: 0.5990
Epoch 2/40
135/135 [==============================] - 7s 50ms/step - loss: 0.9468 - accuracy: 0.6296 - val_loss: 0.9055 - val_accuracy: 0.6302
Epoch 3/40
135/135 [==============================] - 7s 49ms/step - loss: 0.8219 - accuracy: 0.6906 - val_loss: 0.8622 - val_accuracy: 0.6906
Epoch 4/40
135/135 [==============================] - 7s 50ms/step - loss: 0.7391 - accuracy: 0.7266 - val_loss: 0.7227 - val_accuracy: 0.7500
Epoch 5/40
135/135 [==============================] - 7s 51ms/step - loss: 0.6390 - accuracy: 0.7685 - val_loss: 0.6905 - val_accuracy: 0.7615
Epoch 6/40
135/135 [==============================] - 7s 53ms/step - loss: 0.5937 - accuracy: 0.7853 - val_loss: 0.6486 - val_accuracy: 0.7719
Epoch 7/40
135/135 [==============================] - 7s 54ms/step - loss: 0.5589 - accuracy: 0.7991 - val_loss: 0.6188 - val_accuracy: 0.8031

In [178]:
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
test_Y_one_hot = keras.utils.to_categorical(y_test)
test_Y_one_hot = tf.convert_to_tensor(test_Y_one_hot, dtype=tf.float32)


from sklearn.metrics import classification_report

model.evaluate(X_test, test_Y_one_hot)

pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

test_Y_one_hot = np.argmax(test_Y_one_hot, axis=1)

print(pred)
print(test_Y_one_hot)

print(classification_report(test_Y_one_hot, pred, zero_division=0))
print('=========================================')

75/75 [==============================] - 1s 12ms/step
[1 0 3 ... 1 1 5]
[1 0 3 ... 1 1 0]
              precision    recall  f1-score   support

           0       0.81      0.73      0.77       400
           1       0.91      0.95      0.93       400
           2       0.81      0.80      0.80       400
           3       0.80      0.83      0.82       400
           4       0.85      0.83      0.84       400
           5       0.78      0.81      0.79       400

    accuracy                           0.83      2400
   macro avg       0.83      0.83      0.83      2400
weighted avg       0.83      0.83      0.83      2400



In [179]:
test_df = pd.read_csv(os.path.join(PATH, "test_intel-image-classification-csvdata-kaggle.csv"))

test_data = test_data / 255.0

test_data = tf.convert_to_tensor(test_data, dtype=tf.float32)

In [180]:
preds = model.predict(test_data)
predict_test = np.argmax(preds, axis=1)

submission = pd.DataFrame({'number': test_df['number'], 'class': predict_test})
print(submission)
submission.to_csv('submission_cnn_oneHot_01val_40e_64b_maxp_025do_.csv', index=False)

158/158 [==============================] - 2s 12ms/step
      number  class
0          0      1
1          1      0
2          2      0
3          3      0
4          4      0
...      ...    ...
5046    5046      5
5047    5047      5
5048    5048      5
5049    5049      5
5050    5050      5

[5051 rows x 2 columns]
